Introduction and Optimization Problems

Three Types of Models we will go over:
* Optimization
* Statistical
* Simulation

This looks at an Optimization model, a popular Knaosack problem, where you can only eat so many calories, so based on how much you like certain foods, what should you eat in a day. We are trying to maximize the value (taste), while constricting the weight (calories). 

The brute force method is not practical because if there are 100 foods to choose from, then the power set of that size is in the trillions of trillions. 

Greedy Algorithm is a Practical Alternative that when the backpack isn't full, put the best?(taste/calorie, most taste, least calorie) item in the napsack

In [9]:
# Create food class
class Food(object):
    def __init__(self, n, v, w):
        self.name = n
        self.value = v
        self.calories = w

    def getValue(self):
        return self.value

    def getCost(self):
        return self.calories

    def density(self):
        return self.getValue()/self.getCost()

    def __str__(self):
        return f"{self.name}: < {self.value}, {self.calories} >"
    

In [10]:
    # Build a menu of all food names, taste values, and calories
    # This menu will be generated from a list
    
    def buildMenu(names, values, calories):
        """names, values, calories lists of same length.
        name a list of strings
        values and calories lists of numbers
        returns list of Foods"""
        menu = []
        for i in range(len(values)):
            menu.append(Food(names[i], values[i], calories[i]))
        return menu

In [11]:
    #this is essentially the entire algorithm that tries to solve the optimization problem
    def greedy(items, maxCost, keyFunction):
        """Assumes item a list, maxCost >= 0,
        keyFcuntion maps elementns of items to numbers"""
        itemsCopy = sorted(items, key = keyFunction, reverse = True) # keyfunction can be changed

        result = []
        totalValue, totalCost = 0.0 , 0.0

        for i in range(len(itemsCopy)):
            if (totalCost + itemsCopy[i].getCost()) <= maxCost: # check totalcost to see if room in the bag for each item
                result.append(itemsCopy[i]) # if yes append next item to bag
                totalCost += itemsCopy[i].getCost() # add cost to cost counter
                totalValue += itemsCopy[i].getValue() # add value to value counter
        return (result, totalValue)

    # this is called to use the greedy algorithm, but states the constraint, and the key function.
    # the key function is defining the "best" items, either by their high value, low cost, or density (value/cost) 
    def testGreedy(items, constraint, keyFunction):
        taken, val = greedy(items, constraint, keyFunction)
        print('Total value of items taken =', val)
        for item in taken:
            print(' ', item)

    # calling this to test as many greedy alogorithms as we want
    def testGreedys(foods, maxUnits):
        print('Use greedy by value to allocate', maxUnits, 'calories')
        testGreedy(foods, maxUnits, lambda x: x.getValue())
        print('\nUse greedy by cost to allocate', maxUnits, 'calories')
        testGreedy(foods, maxUnits, lambda x: 1/Food.getCost(x))
        print('\nUse greedy by density to allocate', maxUnits, 'calories')
        testGreedy(foods, maxUnits, lambda x: x.density())

In [13]:
names = ['wine', 'beer', 'pizza', 'burger', 'fries',
         'cola', 'apple', 'donut', 'cake']
values = [89,90,95,100,90,79,50,10]
calories = [123,154,258,354,365,150,95,195]
foods = buildMenu(names, values, calories)
testGreedys(foods, 1000)

Use greedy by value to allocate 1000 calories
Total value of items taken = 424.0
  burger: < 100, 354 >
  pizza: < 95, 258 >
  beer: < 90, 154 >
  wine: < 89, 123 >
  apple: < 50, 95 >

Use greedy by cost to allocate 1000 calories
Total value of items taken = 413.0
  apple: < 50, 95 >
  wine: < 89, 123 >
  cola: < 79, 150 >
  beer: < 90, 154 >
  donut: < 10, 195 >
  pizza: < 95, 258 >

Use greedy by density to allocate 1000 calories
Total value of items taken = 413.0
  wine: < 89, 123 >
  beer: < 90, 154 >
  cola: < 79, 150 >
  apple: < 50, 95 >
  pizza: < 95, 258 >
  donut: < 10, 195 >


## Lesson 2

Pros and cons of greedy algorithm. 
* Easy to implement, and computationally efficient.
* Not always the best solution
* How optimal is the algorithm?
* How do we know?
* How close is it to a brute force algorithm?

Search Tree Implementation to solve knapsack problem. It will enumerate all possibilities and give a final value and cost for each combination.

What is the time complexity using a Search Tree. 
* Number of levels = number of items, therefore N + 1
* How many nodes at each level? If level = i then nodes equal 2^i
* Total time complexity is O(2^n + 1), which is exponential. Does this mean brute force method is never useful?

In [15]:
# Brute force method
def maxVal(toConsider, avail):
    """Assumes toConsider a list of items, avail a weight
       Returns a tuple of the total value of a solution to the
         0/1 knapsack problem and the items of that solution"""
    if toConsider == [] or avail == 0:
        result = (0, ()) # base of the recursion, couldn't add anything more
    elif toConsider[0].getCost() > avail: # will the first item fit?
        #Explore right branch only, don't take
        result = maxVal(toConsider[1:], avail) #recusion using the next item in the list, availabilty unchanged
    else: # consider the other branch
        nextItem = toConsider[0]
        #Explore left branch, take the item, and deciding which branch to go from that node
        withVal, withweight = maxVal(toConsider[1:],
                                     avail - nextItem.getCost()) #recursivly run the rest of the items, now with a smaller backpack
        withVal += nextItem.getValue() #increase the value
        #Explore right branch
        withoutVal, withoutweight = maxVal(toConsider[1:], avail)
        #Choose better branch
        if withVal > withoutVal:
            result = (withVal, withweight + (nextItem,))
        else:
            result = (withoutVal, withoutweight)
    return result

#simple program to test search tree
def testMaxVal(foods, maxUnits, printItems = True):
    print('Use search tree to allocate', maxUnits,
          'calories')
    val, taken = maxVal(foods, maxUnits)
    print('Total value of items taken =', val)
    if printItems:
        for item in taken:
            print('   ', item)

names = ['wine', 'beer', 'pizza', 'burger', 'fries',
         'cola', 'apple', 'donut', 'cake']
values = [89,90,95,100,90,79,50,10]
calories = [123,154,258,354,365,150,95,195]
foods = buildMenu(names, values, calories)

testGreedys(foods, 750)
print('')
testMaxVal(foods, 750)

Use greedy by value to allocate 750 calories
Total value of items taken = 284.0
  burger: < 100, 354 >
  pizza: < 95, 258 >
  wine: < 89, 123 >

Use greedy by cost to allocate 750 calories
Total value of items taken = 318.0
  apple: < 50, 95 >
  wine: < 89, 123 >
  cola: < 79, 150 >
  beer: < 90, 154 >
  donut: < 10, 195 >

Use greedy by density to allocate 750 calories
Total value of items taken = 318.0
  wine: < 89, 123 >
  beer: < 90, 154 >
  cola: < 79, 150 >
  apple: < 50, 95 >
  donut: < 10, 195 >

Use search tree to allocate 750 calories
Total value of items taken = 353
    cola: < 79, 150 >
    pizza: < 95, 258 >
    beer: < 90, 154 >
    wine: < 89, 123 >


It worked fine, it was quick, and gave a better result. Issue is there were only 8 items, and 2^8 is not a veryu large number. What if we had a larger menu?

In [16]:
import random

def buildLargeMenu(numItems, maxVal, maxCost):
    items = []
    for i in range(numItems):
        items.append(Food(str(i),
                          random.randint(1, maxVal),
                          random.randint(1, maxCost)))
    return items

for numItems in (5, 10, 15, 20, 25, 30, 35, 40, 45):
   print('Try a menu with', numItems, 'items')
   items = buildLargeMenu(numItems, 90, 250)
   testMaxVal(items, 750, False)  

Try a menu with 5 items
Use search tree to allocate 750 calories
Total value of items taken = 334
Try a menu with 10 items
Use search tree to allocate 750 calories
Total value of items taken = 464
Try a menu with 15 items
Use search tree to allocate 750 calories
Total value of items taken = 524
Try a menu with 20 items
Use search tree to allocate 750 calories
Total value of items taken = 485
Try a menu with 25 items
Use search tree to allocate 750 calories
Total value of items taken = 747
Try a menu with 30 items
Use search tree to allocate 750 calories
Total value of items taken = 859
Try a menu with 35 items
Use search tree to allocate 750 calories
Total value of items taken = 672
Try a menu with 40 items
Use search tree to allocate 750 calories
Total value of items taken = 885
Try a menu with 45 items
Use search tree to allocate 750 calories
Total value of items taken = 630


Is it pointless? Well yes and no. 45 items took a very long time, but with dynamic programing it can be helpful.

Lets look at the finonnaci sequence and see how it relates to our problem. After 35 it goes very slow, which is sort of surprising considering that it not that large of a number. One of the reasons is, with recursion, we are repeating the fin(n) many times over. If we could store the answer in a dictionary and look it up while doing the calcuations, never having to compute it again, it would be much faster. Memoization


In [17]:
def fib(n):
    if n == 0 or n == 1:
        return 1
    else:
        return fib(n - 1) + fib(n - 2)

for i in range(121):
   print('fib(' + str(i) + ') =', fib(i))

fib(0) = 1
fib(1) = 1
fib(2) = 2
fib(3) = 3
fib(4) = 5
fib(5) = 8
fib(6) = 13
fib(7) = 21
fib(8) = 34
fib(9) = 55
fib(10) = 89
fib(11) = 144
fib(12) = 233
fib(13) = 377
fib(14) = 610
fib(15) = 987
fib(16) = 1597
fib(17) = 2584
fib(18) = 4181
fib(19) = 6765
fib(20) = 10946
fib(21) = 17711
fib(22) = 28657
fib(23) = 46368
fib(24) = 75025
fib(25) = 121393
fib(26) = 196418
fib(27) = 317811
fib(28) = 514229
fib(29) = 832040
fib(30) = 1346269
fib(31) = 2178309
fib(32) = 3524578
fib(33) = 5702887
fib(34) = 9227465
fib(35) = 14930352
fib(36) = 24157817
fib(37) = 39088169
fib(38) = 63245986


KeyboardInterrupt: 

In [18]:
# saving walues with dynamic programming
# this returns and very fast results
def fastFib(n, memo = {}):
    """Assumes n is an int >= 0, memo used only by recursive calls
       Returns Fibonacci of n"""
    if n == 0 or n == 1:
        return 1
    try:
        return memo[n] #memo is the dictionary of values memorized, this will be skipped first time through, but called for fastfib(n-1) fastfib(n-2). 
    except KeyError:
        result = fastFib(n-1, memo) + fastFib(n-2, memo)
        memo[n] = result #create new dictionary point with key = n and result = value
        return result

for i in range(121):
   print('fib(' + str(i) + ') =', fastFib(i))

fib(0) = 1
fib(1) = 1
fib(2) = 2
fib(3) = 3
fib(4) = 5
fib(5) = 8
fib(6) = 13
fib(7) = 21
fib(8) = 34
fib(9) = 55
fib(10) = 89
fib(11) = 144
fib(12) = 233
fib(13) = 377
fib(14) = 610
fib(15) = 987
fib(16) = 1597
fib(17) = 2584
fib(18) = 4181
fib(19) = 6765
fib(20) = 10946
fib(21) = 17711
fib(22) = 28657
fib(23) = 46368
fib(24) = 75025
fib(25) = 121393
fib(26) = 196418
fib(27) = 317811
fib(28) = 514229
fib(29) = 832040
fib(30) = 1346269
fib(31) = 2178309
fib(32) = 3524578
fib(33) = 5702887
fib(34) = 9227465
fib(35) = 14930352
fib(36) = 24157817
fib(37) = 39088169
fib(38) = 63245986
fib(39) = 102334155
fib(40) = 165580141
fib(41) = 267914296
fib(42) = 433494437
fib(43) = 701408733
fib(44) = 1134903170
fib(45) = 1836311903
fib(46) = 2971215073
fib(47) = 4807526976
fib(48) = 7778742049
fib(49) = 12586269025
fib(50) = 20365011074
fib(51) = 32951280099
fib(52) = 53316291173
fib(53) = 86267571272
fib(54) = 139583862445
fib(55) = 225851433717
fib(56) = 365435296162
fib(57) = 591286729879
fib(5

So when does this work?
* Optimal substructure: an optimal solution to the overall problem can be constructed from optimal solutions of its subproblems
* Overlapping subproblems: the solution to a subproblem is reused or computed repeatedly, leading to redundant computations.

Does this apply to the napsack problem? well we can write it, but since the nodes are different each time, then it won't speed it up. If there are multiple of the same item, then some nodes will overlap, and therefore dynamic programming will be faster. 

A problem can have overlapping subproblems, even when they have different items in the bad, as long as they are considering between the same next two items, and have the same amount of room left, can it be considered the same problem. 

In [19]:
def fastMaxVal(toConsider, avail, memo = {}):
    """Assumes toConsider a list of subjects, avail a weight
         memo supplied by recursive calls
       Returns a tuple of the total value of a solution to the
         0/1 knapsack problem and the subjects of that solution"""
    if (len(toConsider), avail) in memo:
        result = memo[(len(toConsider), avail)]
    elif toConsider == [] or avail == 0:
        result = (0, ())
    elif toConsider[0].getCost() > avail:
        #Explore right branch only
        result = fastMaxVal(toConsider[1:], avail, memo)
    else:
        nextItem = toConsider[0]
        #Explore left branch
        withVal, withToTake =\
                 fastMaxVal(toConsider[1:],
                            avail - nextItem.getCost(), memo)
        withVal += nextItem.getValue()
        #Explore right branch
        withoutVal, withoutToTake = fastMaxVal(toConsider[1:],
                                                avail, memo)
        #Choose better branch
        if withVal > withoutVal:
            result = (withVal, withToTake + (nextItem,))
        else:
            result = (withoutVal, withoutToTake)
    memo[(len(toConsider), avail)] = result
    return result

def testMaxVal(foods, maxUnits, algorithm, printItems = True):
    print('Menu contains', len(foods), 'items')
    print('Use search tree to allocate', maxUnits,
          'calories')
    val, taken = algorithm(foods, maxUnits)
    if printItems:
        print('Total value of items taken =', val)
        for item in taken:
            print('   ', item)
          
for numItems in (5, 10, 15, 20, 25, 30, 35, 40, 45, 50):
   items = buildLargeMenu(numItems, 90, 250)
   testMaxVal(items, 750, fastMaxVal, True)

Menu contains 5 items
Use search tree to allocate 750 calories
Total value of items taken = 164
    4: < 14, 47 >
    3: < 5, 117 >
    2: < 1, 36 >
    1: < 88, 130 >
    0: < 56, 100 >
Menu contains 10 items
Use search tree to allocate 750 calories
Total value of items taken = 292
    9: < 88, 228 >
    8: < 79, 231 >
    7: < 51, 34 >
    5: < 44, 43 >
    4: < 30, 129 >
Menu contains 15 items
Use search tree to allocate 750 calories
Total value of items taken = 704
    9: < 88, 228 >
    13: < 67, 3 >
    12: < 59, 45 >
    10: < 71, 46 >
    8: < 61, 64 >
    7: < 66, 2 >
    4: < 90, 116 >
    2: < 63, 74 >
    1: < 83, 3 >
    0: < 56, 116 >
Menu contains 20 items
Use search tree to allocate 750 calories
Total value of items taken = 802
    13: < 67, 3 >
    12: < 59, 45 >
    10: < 71, 46 >
    9: < 14, 14 >
    8: < 61, 64 >
    7: < 66, 2 >
    9: < 71, 11 >
    8: < 36, 42 >
    7: < 69, 40 >
    6: < 87, 70 >
    5: < 32, 112 >
    4: < 88, 160 >
    0: < 81, 96 >
Menu cont

How is this so much faster?

Number of possible values to consider is limited by the len(items).  Possible values of avail is closer to the number of distinct sum of weights. 